In [1]:
# This notebook replicates McGill-van Ryzin 2000 Management Science Article

In [92]:
import numpy as np
import scipy.stats as st

In [340]:
# Test problem scenarios: four classes with different fares and demand statistics
n_class = 4
fare = np.array([1050, 950, 699, 520])
mean = np.array([17.3, 45.1, 39.6, 34.0])
std = np.array([5.8, 15.0, 13.2, 11.3])

In [341]:
mean_cumsum = np.cumsum(mean)
std_cumsum = np.sqrt(np.cumsum(std**2))
totalrev = fare * mean
totalrev_cumsum = np.cumsum(totalrev)

In [342]:
(mean_cumsum, std_cumsum, totalrev, totalrev_cumsum)

(array([ 17.3,  62.4, 102. , 136. ]),
 array([ 5.8       , 16.08228839, 20.80576843, 23.67635952]),
 array([18165. , 42845. , 27680.4, 17680. ]),
 array([ 18165. ,  61010. ,  88690.4, 106370.4]))

In [343]:
# Compute weighted average revenue
avgrev_weighted = totalrev_cumsum / mean_cumsum
# Probability of demand being higher than the protection level
prob_emsr = np.array([fare[i+1] / avgrev_weighted[i] for i in range(n_class - 1)])

In [344]:
avgrev_weighted, prob_emsr

(array([1050.        ,  977.72435897,  869.51372549,  782.13529412]),
 array([0.9047619 , 0.71492542, 0.59803541]))

In [345]:
# EMSR-b protection levels
theta_emsr = st.norm.ppf(1 - prob_emsr, mean_cumsum[:-1], std_cumsum[:-1])

In [346]:
theta_emsr

array([ 9.70680404, 53.26796455, 96.83465043])

In [347]:
# Calculate optimal protection level based on stochastic approximation (SA)
ratio = np.array([fare[i] / fare[0] for i in range(1, n_class)])

In [348]:
# In order to find optimal protection levels for SA algorithm, we use
# monte carlo integration specified in McGill-van Ryzin(2004) book
# Page 43
size = 100000
# Normal dist
demand = np.array([[np.random.normal(mean[i], std[i], 1) for i in range(n_class)] for k in range(size)])
demand = demand.reshape(size, n_class)
# Lognormal dist

In [349]:
demand[:10,]

array([[15.87242231, 35.73927795, 33.48018955, 18.9461562 ],
       [16.60951384, 56.2724781 , 56.73055224, 38.7196455 ],
       [13.58374079, 45.74754432, 41.47811538, 43.76904266],
       [19.10569749, 62.06611198, 31.3018971 , 31.76259796],
       [11.27098619, 49.87114527, 52.02159849, 34.49059692],
       [14.76778921, 31.0124776 , 47.63076598, 38.7709801 ],
       [22.77027774, 28.71721543, 28.82716349, 20.1136273 ],
       [20.53327641, 57.68961445, 72.60744561, 36.19261827],
       [17.82351009, 25.13258846, 29.19109205, 58.71143747],
       [ 9.1164696 , 38.07574916, 44.38308885, 44.75031249]])

In [350]:
# Compute random demand partial sums
demand_cumsum = np.array([np.cumsum(demand[i]) for i in range(size)])
demand_cumsum[:10]

array([[ 15.87242231,  51.61170026,  85.09188981, 104.03804601],
       [ 16.60951384,  72.88199194, 129.61254417, 168.33218967],
       [ 13.58374079,  59.33128511, 100.80940049, 144.57844315],
       [ 19.10569749,  81.17180947, 112.47370657, 144.23630453],
       [ 11.27098619,  61.14213146, 113.16372995, 147.65432687],
       [ 14.76778921,  45.78026681,  93.41103279, 132.1820129 ],
       [ 22.77027774,  51.48749317,  80.31465666, 100.42828396],
       [ 20.53327641,  78.22289086, 150.83033647, 187.02295475],
       [ 17.82351009,  42.95609855,  72.1471906 , 130.85862806],
       [  9.1164696 ,  47.19221876,  91.57530761, 136.3256201 ]])

In [353]:
theta_norm = []
cumsum_copy = demand_cumsum.copy()
for j in range(n_class - 1):
    # Step 1
    cumsum_copy_ordered = np.sort(cumsum_copy[:, j])
    # Step 2
    threshold = int(np.floor((fare[j+1] / fare[j]) * len(cumsum_copy_ordered)))
    # An error in the initial algorithm. After calculation l*, there should
    # one more calculation before finding y_j, which is the index used to calculate
    # y_j should be |k|-l*, not l* itself.
    threshold = len(cumsum_copy_ordered) - threshold
    theta = 0.5 * (cumsum_copy_ordered[threshold-1] + cumsum_copy_ordered[threshold])
    # Step 3
    index = np.where(cumsum_copy[:, j] > theta)
    # In step 3, we only keep the updated values we use for next iteration
    cumsum_copy = cumsum_copy[index]
    theta_norm.append(theta)

In [354]:
theta_norm

[9.751898765608983, 53.59980258296807, 98.28777202392007]